# Evolution of maximum radial velocity in the convection zone

In [37]:
%pylab 

Using matplotlib backend: nbAgg
Populating the interactive namespace from numpy and matplotlib


In [38]:
import nugridpy.utils as utils
from ppmpy import ppm  
import numpy as np
import matplotlib.pyplot as plt
%matplotlib nbagg

def get_vr_max_evolution(prof, cycles, r1, r2):
    r = prof.get('Y', fname = cycles[0], resolution = 'l')
    idx1 = np.argmin(np.abs(r - r1))
    idx2 = np.argmin(np.abs(r - r2))

    t = np.zeros(len(cycles))
    vr_max = np.zeros(len(cycles))
    for k in range(len(cycles)):
        t[k] = prof.get('t', fname = cycles[k], resolution = 'l')[-1]
        vr_rms  = prof.get('EkY', fname = cycles[k], resolution = 'l')**0.5
        vr_max[k] = np.max(vr_rms[idx2:idx1])
    
    return t, vr_max

In [39]:
data_dir='/data/ppm_rpod2/YProfiles/O-shell-M25/'

In [40]:
prof = ppm.yprofile(data_dir+'D1')

Reading attributes from file  YProfile-01-0348.bobaaa
Analyzing headers ...
There are 349 YProfile files in the /data/ppm_rpod2/YProfiles/O-shell-M25/D1 directory.
Ndump values range from 0 to 348
Time values range from 0.0 to 3480.0


In [ ]:
sparse = 1
markevery = 25
Ymin=4.; Ymax=8.
cases = ['D1','D2']
for case in cases:
    cycles = range(prof.cycles[0], prof.cycles[-1], sparse)
    prof = yprofile(data_dir+case)
    t, vr_max = get_vr_max_evolution(prof, cycles, Ymin, Ymax)
    plt.plot(t/60.,  1e3*vr_max,  color = cb(0),\
             marker = 's', markevery = markevery, label = case)
    

In [43]:
from nugridpy import utils as ut

In [47]:
ut.linestylecb(3)


(':', '>', (0.37254901960784315, 0.6196078431372549, 0.8196078431372549), 5)

In [ ]:
cycles_D2 = range(0, 164, sparse)
prof_D2 = yprofile('/ppm_data1/rpod2/YProfiles/O-shell-M25/D2')
t_D2, vr_max_D2 = get_vr_max_evolution(prof_D2, cycles_D2, 4., 8.)

cycles_D8 = range(0, 218, sparse)
prof_D8 = yprofile('/ppm_data1/rpod2/YProfiles/O-shell-M25/D8')
t_D8, vr_max_D8 = get_vr_max_evolution(prof_D8, cycles_D8, 4., 8.)

cycles_D5 = range(0, 224, sparse)
prof_D5 = yprofile('/ppm_data1/rpod2/YProfiles/O-shell-M25/D5')
t_D5, vr_max_D5 = get_vr_max_evolution(prof_D5, cycles_D5, 4., 8.)

cycles_D6 = range(0, 249, sparse)
prof_D6 = yprofile('/ppm_data1/rpod2/YProfiles/O-shell-M25/D6')
t_D6, vr_max_D6 = get_vr_max_evolution(prof_D6, cycles_D6, 4., 8.)

cycles_D9 = range(0, 262, sparse)
prof_D9 = yprofile('/ppm_data1/rpod2/YProfiles/O-shell-M25/D9')
t_D9, vr_max_D9 = get_vr_max_evolution(prof_D9, cycles_D9, 4., 8.)

cycles_D10 = range(0, 263, sparse)
prof_D10 = yprofile('/ppm_data1/rpod2/YProfiles/O-shell-M25/D10')
t_D10, vr_max_D10 = get_vr_max_evolution(prof_D10, cycles_D10, 4., 8.)

In [ ]:
cb = utils.colourblind
ifig = 1; plt.close(ifig); plt.figure(ifig)
plt.plot(t_D1/60.,  1e3*vr_max_D1,  color = cb(0), marker = 's', markevery = markevery, label = 'D1')
plt.plot(t_D2/60.,  1e3*vr_max_D2,  color = cb(1), marker = 'D', markevery = markevery, label = 'D2')
plt.plot(t_D8/60.,  1e3*vr_max_D8,  color = cb(2), marker = 'd', markevery = markevery, label = 'D8')
plt.plot(t_D5/60.,  1e3*vr_max_D5,  color = cb(3), marker = 'v', markevery = markevery, label = 'D5')
plt.plot(t_D6/60.,  1e3*vr_max_D6,  color = cb(4), marker = '^', markevery = markevery, label = 'D6')
plt.plot(t_D9/60.,  1e3*vr_max_D9,  color = cb(5), marker = '>', markevery = markevery, label = 'D9')
plt.plot(t_D10/60., 1e3*vr_max_D10, color = cb(6), marker = '<', markevery = markevery, label = 'D10')
plt.xlim((0., 60.))
plt.ylim((0., 130.))
plt.xlabel('t / min')
plt.ylabel(r'v$_r$ / km s$^{-1}$')
plt.legend(loc = 0)